In [1]:
import os.path
import numpy as np
import pandas as pd
from time import time
from os import listdir
from os.path import isfile, join

In [2]:
def file_csv_list(target_path):
    return [f for f in listdir(target_path) if isfile(join(target_path, f))]

In [3]:
def read_csv(filename):
    # read revise csv file and print cost time
    t0 = time()
    df = pd.read_csv(filename, error_bad_lines=False)
    print("time for read csv: %.2f" % (time()-t0))
    return df

In [4]:
def remove_date_column(df):
    # we don't use VOICE_DATE/VOICE_MONTH bcz it is a monthly data
    if("VOICE_DATE" in df.columns):
        df = df.drop('VOICE_DATE', 1)
        print("Drop VOICE_DATE")
    if("VOICE_MONTH" in df.columns):
        df = df.drop('VOICE_MONTH', 1)
        print("Drop VOICE_MONTH")
    return df

In [5]:
def save_dataframe(df, out_filename):
   # write to csv and no index
    t0 = time()
    # aggr_df.to_csv(out_filename + ".csv", index=False, encoding='utf-8')
    df.to_csv(out_filename + ".csv", encoding='utf-8')
    print("time for output csv file: %.2f" % (time()-t0))

In [6]:
# set configure
# path = "../DATA_FULL/"
path = "./CDR_MONTHLY/"
# filename = "CDR_MO_HOUR_revise_month_1"
# relative_filename = path + filename + ".csv"
out_path = "./CDR_MONTHLY_AGGR/"
for filename in file_csv_list(path):
    if ".csv" in filename:
        filename_none_postfix = filename[:-4]
        out_filename =  out_path + filename_none_postfix + "_aggregate"
        df = read_csv(path + filename)
        df = remove_date_column(df)
        df = df.groupby('MINING_DW_SUBSCR_NO').sum()
        save_dataframe(df, out_filename)

C:\Users\VIPLAB\AppData\Local\conda\conda\envs\py36\lib\site-packages\IPython\core\interactiveshell.py:2821: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


time for read csv: 201.93
Drop VOICE_DATE
time for output csv file: 93.44
